In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
warnings.simplefilter(action="ignore")

from statsmodels.stats.weightstats import ttest_ind
from scipy import stats
from sklearn import metrics

%matplotlib inline

## Load and Check Primary Dataset 
---

In [2]:
df = pd.read_stata("maindataset_thesis.dta")
df = df.query('tk01 != 5 and tk01 != 7') # exclude the retired
df.sample(5)

,sex,respondent,age,marstat,pidlink,ar02b,hhid07,pid07,hhid00,pid00,...,E_hr1type_t1,F_hr1type_t1,IHK_t1,IHK,nmkab2014,hhid14,pid14,_2014,married,single
8378,1,3,18.0,single,267010010,13,2670100,10.0,,NaN,...,11000001.00,1500001.00,77.80,116.27,DENPASAR,2670100,10.0,1,0,1
2660,0,3,15.0,single,195070005,3,1950731,2.0,1950731,2.0,...,1.66,1159201.28,41.94,77.88,TEGAL,,NaN,0,0,1
2051,1,3,24.0,single,145130004,3,1451300,4.0,1451300,4.0,...,1.00,801.00,39.00,76.35,SERANG,,NaN,0,0,1
7712,0,3,17.0,single,216200012,3,2162000,12.0,,NaN,...,11000001.00,1000001.00,77.88,113.55,BREBES,2162000,12.0,1,0,1
8592,1,3,21.0,single,279160009,3,2791600,9.0,,NaN,...,30000001.00,1500001.00,73.87,120.90,LOMBOK BARAT,2791600,9.0,1,0,1


In [3]:
df.shape

(9348, 48)

In [4]:
df.columns

Index(['sex', 'respondent', 'age', 'marstat', 'pidlink', 'ar02b', 'hhid07',
       'pid07', 'hhid00', 'pid00', 'tk02', 'tk03', 'tk04', 'tk01a', 'tk01b',
       'tk01c', 'tk01d', 'tk16d', 'tk16i', 'tk01', 'tk25a9x', 'tk16h', 'main',
       'NEET', 'NEET__just_other_', 'year', 'nonfood_t1', 'food_t1', 'pce_t1',
       'kabid_t1', 'nonfood_t', 'food_t', 'pce_t', 'kabid', 'A_hr1type_t',
       'E_hr1type_t', 'F_hr1type_t', 'A_hr1type_t1', 'E_hr1type_t1',
       'F_hr1type_t1', 'IHK_t1', 'IHK', 'nmkab2014', 'hhid14', 'pid14',
       '_2014', 'married', 'single'],
      dtype='object')

In [5]:
df["tk01"].value_counts()

3     3963
1     3197
4     1217
95     544
2      427
Name: tk01, dtype: int64

In [6]:
a = df.query('tk01 == 4 & NEET == 1')
b = df.query('tk01 == 95 & NEET == 1')
c = df.query('tk01 == 2 & NEET == 1')
d = df.query('tk01 == 1 & NEET == 0')
e = df.query('tk01 == 3 & NEET == 0')

In [7]:
df1 = pd.concat([a,b,c,d,e])
z = df1.index.tolist()

In [8]:
#make sure that main activity in line with our NEET definition
w = []
for i in df.index.tolist():
    if i not in z: # check wheter the main activity should be NEET but they have more than one activity which non-NEET
        w.append(i)

In [9]:
df2 = df.loc[w]
df2["tk01"] = df2[["tk01a", "tk02", "tk03", "tk04"]].sum(axis=1).apply(lambda x: 3 if x == 0 else 1) # "tk01b", "tk01c", "tk01d", "tk25a9x"

# I can't control the mutually exclusiveness within this group
print ("There are", len(df2), "samples based on main activity ALONE should classify to NEET yet actually not--based on series of activity") 

There are 561 samples based on main activity ALONE should classify to NEET yet actually not--based on series of activity


In [10]:
data = pd.concat([df1, df2])
data.sample(5)

,sex,respondent,age,marstat,pidlink,ar02b,hhid07,pid07,hhid00,pid00,...,E_hr1type_t1,F_hr1type_t1,IHK_t1,IHK,nmkab2014,hhid14,pid14,_2014,married,single
3040,1,3,17.0,single,227100004,3,2271000,4.0,2271000,4.0,...,1.0,700001.0,43.27,78.68,MALANG,,NaN,0,0,1
566,0,3,20.0,married,041140003,3,0411400,3.0,0411400,3.0,...,3.0,50001.0,37.62,78.64,OGAN KOMERING ULU TIMUR,,NaN,0,1,0
9157,1,3,16.0,single,307070009,3,3070700,9.0,,NaN,...,46000001.0,11500001.0,78.37,124.29,BULUKUMBA,3070700,9.0,1,0,1
8695,1,3,24.0,married,283150003,3,2831500,3.0,,NaN,...,450001.0,1500001.0,73.87,120.90,LOMBOK TIMUR,2831500,3.0,1,1,0
9034,0,3,22.0,married,299290006,3,2992900,6.0,,NaN,...,600001.0,1000001.0,75.08,118.42,HULU SUNGAI TENGAH,2992900,6.0,1,1,0


## Gathering other control variables
---

### 2007

In [11]:
path07 = "C:/Users/Redata/Downloads/read/kemiskinan/hh2007/"
a = pd.read_stata(path07 + "pce07nom.dta")[["hhid07", "hhsize"]]
b = pd.read_stata(path07 + "bk_sc.dta")[["hhid07", "sc05", "sc21x"]]
c = pd.merge(a,b, left_on="hhid07", right_on="hhid07", how="outer")
e = pd.merge(data[data["year"] == 2007], c, left_on="hhid07", right_on="hhid07", how="left")

In [12]:
sw = pd.read_stata(path07 +"b3a_sw.dta")[["pidlink", "sw01", "sw03b"]]
kk = pd.read_stata(path07 +"b3b_kk1.dta")[["pidlink", "kk01", "kk02a", "kk02c"]]

#education vairables
educ1 = pd.read_stata(path07 +"b3a_dl2.dta")[["dl2type", "pidlink", "dl16j", "dl16ja"]]
educ1_t = pd.merge(educ1.groupby(["pidlink"], as_index=False)["dl2type"].max(), 
         educ1, left_on=["pidlink", "dl2type"], right_on=["pidlink", "dl2type"])

educ2 = pd.read_stata(path07 +"b3a_dl1.dta")[["pidlink", "dl06", "dl07", "dl05b"]]

educ = pd.merge(educ1_t, educ2, left_on="pidlink", right_on="pidlink")

#the distance (in minutes) to school
#convert to minutes
educ.dl16ja = educ.dl16ja.apply(lambda x:60 if x == 2 else x) #change the a hour measurment
educ["dl16j"] = educ["dl16j"] * educ["dl16ja"]

#general health
health1 = pd.read_stata(path07 +"bus1_1.dta")[[ "pidlink", "us06"]]
health2 = pd.read_stata(path07 +"bus1_2.dta")[["pidlink", "us04"]]

#logical test
logic = pd.read_stata(path07 +"bek_ek2.dta", index_col="pidlink")[["ek1x", "ek2x", "ek3x", "ek4x", "ek5x", "ek6x", "ek7x", "ek8x",
                                             "ek9x", "ek10x", "ek11x", "ek12x", "ek13x", "ek14x", "ek15x", "ek16x", 
                                              "ek17x", "ek18x","ek19x", "ek20x"]]
for i in logic.columns:
    logic[i] = logic[i].apply(lambda x: 0 if x != 1 else 1)

logic["test"] = logic.sum(axis=1)/20
logic.reset_index(inplace=True)

### Additional variable

In [13]:
read = pd.read_stata( path07+ "b3a_dl1.dta")[["pidlink", "dl02", "dl03"]] 
#migra = pd.read_stata( path07+ "b3a_mg2.dta")[["pidlink", "mg27x", "mg36"]]
preference = pd.read_stata( path07+ "b3a_si.dta")[["pidlink", "si21b", "si22b", "si21a"]]
village = pd.read_stata( path07+ "b3a_tr.dta")[["pidlink", "tr06", "tr11"]]
tobaco = pd.read_stata( path07 + "b3b_km.dta")[["pidlink", "km01a"]]
election = pd.read_stata( path07+ "b3b_pm1.dta")[["pidlink", "pm24a", "pm24b", "pm24c","pm24d","pm24e","pm24f","pm24g","pm24h"]]

In [14]:
df = sw.copy()
for i in [kk, educ, health1, health2, logic[["test", "pidlink"]], read, preference, village, tobaco, election]:
    df = pd.merge(df, i, left_on="pidlink", right_on="pidlink", how="outer")

In [15]:
def educagg(data):
    one = [2, 11, 90, 72] #primary school
    two = [3, 4, 12, 73] #junior high school
    three = [5, 6, 15, 74] #high school
    four = [60, 61, 62, 63, 13] #college
    if data in one:
        return 1
    elif data in two:
        return 2
    elif data in three:
        return 3
    elif data in four:
        return 4
    else:
        return np.nan
    
df["educ"] = df["dl06"].apply(lambda x: educagg(x)) #aggregation
df["educ"] = df["educ"].fillna(df["dl2type"]) #fillna with another variable

In [16]:
data2007 = pd.merge(e, df, left_on="pidlink", right_on="pidlink", how="left")

### 2014

In [17]:
path14 = "C:/Users/Redata/Downloads/read/kemiskinan/hh2014/"
r = pd.read_stata(path14 + "pce14nom.dta")[["hhid14", "hhsize"]]
s = pd.read_stata(path14 + "bk_sc1.dta")[["hhid14", "sc05", "sc21x"]]
t = pd.merge(r,s, left_on="hhid14", right_on="hhid14", how="outer")
u = pd.merge(data[data["year"] == 2014], t, left_on="hhid14", right_on="hhid14", how="left")

In [18]:
sw = pd.read_stata(path14 +"b3a_sw.dta")[["pidlink", "sw01", "sw03b"]]
kk = pd.read_stata(path14 +"b3b_kk1.dta")[["pidlink", "kk01", "kk02a", "kk02c"]]


#education vairables
educ1 = pd.read_stata(path14 +"b3a_dl2.dta")[["dl2type", "pidlink", "dl16j", "dl16ja"]]
educ1_t = pd.merge(educ1.groupby(["pidlink"], as_index=False)["dl2type"].max(), 
         educ1, left_on=["pidlink", "dl2type"], right_on=["pidlink", "dl2type"])

educ2 = pd.read_stata(path14 +"b3a_dl1.dta")[["pidlink", "dl06", "dl07", "dl05b"]]

educ = pd.merge(educ1_t, educ2, left_on="pidlink", right_on="pidlink")

#the distance (in minutes) to school
#convert to minutes
educ.dl16ja = educ.dl16ja.astype("str").str.extract(r'(\d+)')[0].apply(lambda x: 0 if x != "1" else 1).astype("float")
educ.dl16ja = educ.dl16ja.apply(lambda x:60 if x == 2 else x) #change the a hour measurment
educ["dl16j"] = educ["dl16j"].astype("float") * educ["dl16ja"]

#general health
health = pd.read_stata(path14 +"bus_us.dta")[[ "pidlink", "us06", "us04"]]

#logical test
logic = pd.read_stata(path14 +"ek_ek2.dta", index_col="pidlink")[["ek1_ans", "ek2_ans", "ek3_ans", "ek4_ans", "ek5_ans", "ek6_ans",
                                             "ek7_ans", "ek8_ans", "ek9_ans", "ek10_ans", "ek11_ans", "ek12_ans", 
                                             "ek13_ans", "ek14_ans", "ek15_ans", "ek16_ans", "ek17_ans", "ek18_ans",
                                             "ek19_ans", "ek20_ans", "ek21_ans", "ek22_ans"]]
for i in logic.columns:
    logic[i] = logic[i].astype("str").str.extract(r'(\d+?)')[0].astype("float").apply(lambda x: 0 if x != 1 else 1)

logic["test"] = logic.sum(axis=1)/22
logic.reset_index(inplace=True)

### Additional variable

In [19]:
read = pd.read_stata( path14+ "b3a_dl1.dta")[["pidlink", "dl02", "dl03"]] 
#migra = pd.read_stata( path14+ "b3a_mg2.dta")[["pidlink", "mg27x", "mg36"]]
preference = pd.read_stata( path14+ "b3a_si.dta")[["pidlink", "si21b", "si22b", "si21a"]]
village = pd.read_stata( path14+ "b3a_tr.dta")[["pidlink", "tr06", "tr11"]]
tobaco = pd.read_stata( path14 + "b3b_km.dta")[["pidlink", "km01a"]]
election = pd.read_stata( path14+ "b3b_pm1.dta")[["pidlink", "pm24a", "pm24b", "pm24c","pm24d","pm24e","pm24f","pm24g","pm24h"]]

In [20]:
df = sw.copy()
for i in [kk, educ, health, logic[["test", "pidlink"]], read, preference, village, tobaco, election]:
    df = pd.merge(df, i, left_on="pidlink", right_on="pidlink", how="outer")

In [21]:
df["educ"] = df["dl06"].apply(lambda x: educagg(x)) #aggregation
df["educ"] = df["educ"].fillna(df["dl2type"]) #fillna with another variable

In [22]:
data2014 = pd.merge(u, df, left_on="pidlink", right_on="pidlink", how="left")

## Concat both year

In [23]:
data = pd.concat([data2007, data2014])
data.sample(10)

,sex,respondent,age,marstat,pidlink,ar02b,hhid07,pid07,hhid00,pid00,...,km01a,pm24a,pm24b,pm24c,pm24d,pm24e,pm24f,pm24g,pm24h,educ
2295,0,3,15.0,single,167180008,3,1671331,5.0,1671800,8.0,...,3,6,6,6,6,6,6,6,6,2
665,1,3,20.0,single,057240007,3,0572411,3.0,0572400,7.0,...,1,6,6,6,6,6,6,6,6,3
3012,1,3,20.0,single,290270003,3,2902700,3.0,2902700,3.0,...,1,6,6,6,6,6,6,6,6,3
703,0,3,24.0,single,307070005,3,3070700,5.0,,NaN,...,3:No,1:Yes,1:Yes,1:Yes,1:Yes,1:Yes,1:Yes,1:Yes,3:No,4:D1
103,0,3,19.0,single,045260008,14,0452600,8.0,0452600,8.0,...,3,6,6,6,6,6,6,6,6,1
1323,0,3,24.0,single,016160006,13,0161631,4.0,0161631,4.0,...,3,1,1,1,1,1,6,6,6,4
3013,0,3,22.0,single,290420003,3,2904200,3.0,2904200,3.0,...,3,1,1,1,1,1,3,3,1,4
4068,0,3,17.0,single,270050010,3,2700500,10.0,,NaN,...,3:No,1:Yes,3:No,3:No,3:No,3:No,3:No,3:No,6:Not applicable,3:Senior high
3343,1,3,15.0,single,155040008,4,1550441,3.0,,NaN,...,3:No,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,2:Junior high
4667,0,3,16.0,single,100200008,10,1002000,8.0,,NaN,...,3:No,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,6:Not applicable,3:Senior high


## Cleaning Data
---

In [24]:
col_to_clean = ["educ", "dl05b", "dl07", "dl06", "dl2type", "kk02c", "kk01", "sw03b", "sw01", "sc21x", 
               "dl02", "dl03", "si21b", "si22b", "si21a", "sc05",
                "tr06", "tr11", "km01a", "pm24a", "pm24b", "pm24c","pm24d","pm24e","pm24f","pm24g","pm24h"]

for i in col_to_clean:
    data[i] = data[i].astype("str").str.extract(r'(\d+?)')[0].astype("float")

In [25]:
binar = ["sc05", "dl02", "dl03", "si21b", "si22b", "si21a", 
         "km01a", "pm24a", "pm24b", "pm24c","pm24d","pm24e","pm24f","pm24g","pm24h"]
for i in binar:
    data[i] = data[i].astype("float").apply(lambda x: np.nan if x in [8, 9] else x).astype("float")
    
for i in binar:
    data[i] = data[i].astype("float").apply(lambda x: 0 if x not in [1, np.nan] else x).astype("float")

In [26]:
data.to_stata("dataraw_thesis_redi.dta")